In [30]:
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout

In [31]:
list_of_dataframes = []
for file in glob.glob("/home/haohao/Desktop/Hieu-cicids2017/dataset/*"):
    list_of_dataframes.append(pd.read_csv(file))
df = pd.concat(list_of_dataframes).reset_index(drop=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [32]:
df.head(10)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
5,22,244,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
6,22,1307239,41,40,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
7,60058,82,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
8,22,171,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
9,22,210,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [33]:
dropList = []
cnt = 0
for i in range(len(df)):
  if df.iloc[i, df.columns.get_loc(' Label')] == 'BENIGN':
    if cnt < 2043097:
      dropList.append(i)
    cnt += 1

df = df.drop(dropList)

In [34]:
y = pd.get_dummies(df[' Label']).to_numpy()
y[0:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=uint8)

In [35]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)
scaler = MinMaxScaler()
X = df.iloc[:, :-1]
X = scaler.fit_transform(X)

In [36]:
X[0:10]

array([[1.22161650e-03, 4.18421674e-02, 5.02512563e-04, 3.38776340e-04,
        2.45280188e-04, 4.80952381e-05, 1.52397260e-02, 0.00000000e+00,
        1.97221821e-02, 2.57517453e-02, 6.04281768e-02, 0.00000000e+00,
        4.87544177e-02, 5.57587013e-02, 5.76389127e-03, 4.00000438e-01,
        4.18422413e-03, 1.85168732e-02, 4.13804913e-02, 2.66666638e-07,
        4.61675000e-04, 9.23350000e-05, 2.10426129e-04, 3.48858333e-04,
        2.58333308e-07, 4.18410667e-02, 1.04602667e-02, 3.10551502e-02,
        4.17094417e-02, 8.77500000e-06, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.04706846e-03, 5.69144251e-04,
        3.98322346e-07, 4.97902933e-07, 0.00000000e+00, 4.49486301e-02,
        7.73840672e-02, 6.70970307e-02, 4.49945989e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        7.36809133e-02, 1.97221821e-02, 4.87544177e-02, 1.047068

In [37]:
y_labels = np.argmax(y, axis=1)

In [39]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Tạo đối tượng SMOTE
smote = SMOTE()

# Áp dụng SMOTE
X_resampled, y_resampled = smote.fit_resample(X, y_labels)



Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f56db8b5dc0>
Traceback (most recent call last):
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modul

In [45]:
num_classes = np.unique(y_resampled).size

# Convert back to one-hot encoding
y_resampled_one_hot = np.eye(num_classes)[y_resampled]

In [46]:
y_resampled_one_hot[:3]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [40]:

# Đếm số lượng mỗi nhãn
label_counts = np.bincount(y_resampled)

# In ra số lượng mỗi nhãn
for i, count in enumerate(label_counts):
    print(f"Label {i}: {count} occurrences")

Label 0: 231073 occurrences
Label 1: 231073 occurrences
Label 2: 231073 occurrences
Label 3: 231073 occurrences
Label 4: 231073 occurrences
Label 5: 231073 occurrences
Label 6: 231073 occurrences
Label 7: 231073 occurrences
Label 8: 231073 occurrences
Label 9: 231073 occurrences
Label 10: 231073 occurrences
Label 11: 231073 occurrences
Label 12: 231073 occurrences
Label 13: 231073 occurrences
Label 14: 231073 occurrences


In [7]:

# Tạo đối tượng RandomUnderSampler
rus = RandomUnderSampler()

# Áp dụng RandomUnderSampler
X_resampled, y_resampled = rus.fit_resample(X, y_labels)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled_one_hot, test_size=0.2, random_state=75)

In [48]:
y_train.shape

(2772876, 15)

In [49]:
num_classes = y_train.shape[1]

def build_model():
  model = Sequential()
  model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.3))
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(num_classes, activation='softmax'))  # Sử dụng softmax cho phân loại đa lớp
  model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),
                loss='categorical_crossentropy',  # Sử dụng categorical_crossentropy cho phân loại đa lớp
                metrics=['accuracy'])
  return model



In [50]:
model = build_model()

In [51]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,  # Sử dụng một phần của dữ liệu huấn luyện để làm dữ liệu kiểm thử
                    verbose=1)


2024-01-12 08:24:45.112162: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 692109600 exceeds 10% of free system memory.


Epoch 1/10
69322/69322 [==============================] - 784s 11ms/step - loss: 0.4316 - accuracy: 0.8272 - val_loss: 0.2434 - val_accuracy: 0.8826
Epoch 2/10
69322/69322 [==============================] - 620s 9ms/step - loss: 0.2529 - accuracy: 0.8804 - val_loss: 0.2196 - val_accuracy: 0.8892
Epoch 3/10
69322/69322 [==============================] - 625s 9ms/step - loss: 0.2305 - accuracy: 0.8871 - val_loss: 0.2008 - val_accuracy: 0.8963
Epoch 4/10
69322/69322 [==============================] - 628s 9ms/step - loss: 0.2172 - accuracy: 0.8916 - val_loss: 0.1909 - val_accuracy: 0.9011
Epoch 5/10
69322/69322 [==============================] - 631s 9ms/step - loss: 0.2076 - accuracy: 0.8955 - val_loss: 0.1891 - val_accuracy: 0.9036
Epoch 6/10
69322/69322 [==============================] - 634s 9ms/step - loss: 0.2006 - accuracy: 0.8984 - val_loss: 0.1822 - val_accuracy: 0.9054
Epoch 7/10
69322/69322 [==============================] - 635s 9ms/step - loss: 0.1952 - accuracy: 0.9008 - val

In [52]:
model.save('cicids2017.h5')

/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [53]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Test Loss: 0.17934498190879822, Test Accuracy: 0.9045554399490356


In [54]:
from sklearn.metrics import classification_report, roc_auc_score

def eval(model):
  # Dự đoán nhãn trên tập kiểm thử
  y_pred = model.predict(X_test)

  # Chuyển đổi dự đoán thành nhãn cụ thể
  y_pred_labels = np.argmax(y_pred, axis=1)
  y_true_labels = np.argmax(y_test, axis=1)

  # Tính toán F1-score, Recall và Precision
  report = classification_report(y_true_labels, y_pred_labels, target_names=None)
  print(report)


In [55]:
eval(model)

21664/21664 [==============================] - 51s 2ms/step
              precision    recall  f1-score   support

           0       1.00      0.92      0.96     46425
           1       0.98      1.00      0.99     46410
           2       0.98      0.99      0.99     46215
           3       1.00      0.98      0.99     45966
           4       0.98      0.99      0.99     45814
           5       0.99      0.99      0.99     46341
           6       0.99      0.99      0.99     45832
           7       1.00      0.99      0.99     46257
           8       1.00      1.00      1.00     45806
           9       1.00      1.00      1.00     46331
          10       1.00      1.00      1.00     46387
          11       0.68      0.99      0.81     46399
          12       0.97      0.14      0.25     46531
          13       0.95      0.60      0.74     46225
          14       0.55      0.99      0.70     46280

    accuracy                           0.90    693219
   macro avg       0

In [56]:
# Chia tập huấn luyện thành 10 phần cho mô hình hóa federated learning
client_data = np.array_split(X_train, 10)
client_labels = np.array_split(y_train, 10)


In [62]:
global_model = build_model()


In [60]:
def fed_avg(models):
    # Initialize average weights
    avg_weights = [np.zeros_like(w) for w in models[0].get_weights()]

    # Sum weights from all models
    for model in models:
        for idx, layer_weights in enumerate(model.get_weights()):
            avg_weights[idx] += layer_weights

    # Divide by the number of models to get the average
    avg_weights = [w / len(models) for w in avg_weights]

    return avg_weights


In [63]:
num_rounds = 10
num_epochs = 3
batch_size = 32

for round in range(num_rounds):
    local_models = []
    cnt = 1
    for client_X, client_y in zip(client_data, client_labels):
        print('Round ', round, ' - Client ', cnt)
        cnt += 1
        # Sao chép trọng số từ global model
        local_model = build_model()
        local_model.set_weights(global_model.get_weights())

        # Huấn luyện mô hình cục bộ
        local_model.fit(client_X, client_y, epochs=num_epochs, batch_size=batch_size, verbose=0)
        local_models.append(local_model)

    # Cập nhật global model dựa trên trung bình trọng số từ các mô hình cục bộ
    global_model.set_weights(fed_avg(local_models))

    print('Round ', round, ' - Evaluation:')
    eval(global_model)

Round  0  - Client  1
Round  0  - Client  2
Round  0  - Client  3
Round  0  - Client  4
Round  0  - Client  5
Round  0  - Client  6
Round  0  - Client  7
Round  0  - Client  8
Round  0  - Client  9
Round  0  - Client  10
Round  0  - Evaluation:
21664/21664 [==============================] - 63s 3ms/step
              precision    recall  f1-score   support

           0       0.99      0.82      0.90     46425
           1       0.93      0.99      0.96     46410
           2       0.92      0.99      0.95     46215
           3       0.98      0.96      0.97     45966
           4       0.93      0.97      0.95     45814
           5       0.95      0.93      0.94     46341
           6       0.96      0.84      0.90     45832
           7       0.91      1.00      0.95     46257
           8       1.00      1.00      1.00     45806
           9       0.98      0.96      0.97     46331
          10       1.00      0.99      0.99     46387
          11       0.62      0.99      0.76   